# Open Reading Frames

Problem
Either strand of a DNA double helix can serve as the coding strand for RNA transcription. Hence, a given DNA string implies six total reading frames, or ways in which the same region of DNA can be translated into amino acids: three reading frames result from reading the string itself, whereas three more result from reading its reverse complement.

An open reading frame (ORF) is one which starts from the start codon and ends by stop codon, without any other stop codons in between. Thus, a candidate protein string is derived by translating an open reading frame into amino acids until a stop codon is reached.

Given: A DNA string s of length at most 1 kbp in FASTA format.

Return: Every distinct candidate protein string that can be translated from ORFs of s. Strings can be returned in any order.

Sample Dataset
```
>Rosalind_99
AGCCATGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAG
Sample Output
MLLGSFRLIPKETLIQVAGSSPCNLS
M
MGMTPRLGLESLLE
MTPRLGLESLLE
```

In [36]:
sample_seq = "AGCCATGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAG"

Encontrei a seguinte tabela com o código genético padrão em um site do NCBI. Salvei-a num arquivo na pasta `data`.

In [36]:
raw_genetic_code_path = "data/GeneticCode.txt"
print(read(raw_genetic_code_path, String))

#  Source: https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi?chapter=tgencodes#SG1
TTT F Phe      TCT S Ser      TAT Y Tyr      TGT C Cys  
TTC F Phe      TCC S Ser      TAC Y Tyr      TGC C Cys  
TTA L Leu      TCA S Ser      TAA * Ter      TGA * Ter  
TTG L Leu i    TCG S Ser      TAG * Ter      TGG W Trp  

CTT L Leu      CCT P Pro      CAT H His      CGT R Arg  
CTC L Leu      CCC P Pro      CAC H His      CGC R Arg  
CTA L Leu      CCA P Pro      CAA Q Gln      CGA R Arg  
CTG L Leu i    CCG P Pro      CAG Q Gln      CGG R Arg  

ATT I Ile      ACT T Thr      AAT N Asn      AGT S Ser  
ATC I Ile      ACC T Thr      AAC N Asn      AGC S Ser  
ATA I Ile      ACA T Thr      AAA K Lys      AGA R Arg  
ATG M Met i    ACG T Thr      AAG K Lys      AGG R Arg  

GTT V Val      GCT A Ala      GAT D Asp      GGT G Gly  
GTC V Val      GCC A Ala      GAC D Asp      GGC G Gly  
GTA V Val      GCA A Ala      GAA E Glu      GGA G Gly  
GTG V Val      GCG A Ala      GAG E Glu      GGG G G

A seguir, lemos e guardamos o conteúdo do arquivo em um dicionário.

In [34]:
genetic_code = Dict{String, String}()

# itere pelas linhas do arquivo
for line in readlines(raw_genetic_code_path)
    # se a linha não (estiver vazia ou for um comentário)
    if !(line == "" || startswith(line, '#'))
        columns = split(strip(line), r"\s+(i\s*)?")  # pula o 'i' que aparece de vez em quando
        # o arquivo tem apenas 3 colunas que se repetem na horizontal
        for i in 1:3:10
            genetic_code[columns[i]] = columns[i+1]
        end
    end
end

In [35]:
genetic_code

Dict{String, String} with 64 entries:
  "TGT" => "C"
  "GAC" => "D"
  "TTC" => "F"
  "TAG" => "*"
  "GCT" => "A"
  "CCT" => "P"
  "GTG" => "V"
  "GGC" => "G"
  "CGG" => "R"
  "ATT" => "I"
  "GAT" => "D"
  "CAG" => "Q"
  "ATG" => "M"
  "CTC" => "L"
  "TCT" => "S"
  "CGT" => "R"
  "ACG" => "T"
  "AGA" => "R"
  "TGG" => "W"
  "TCG" => "S"
  "TTA" => "L"
  "AGT" => "S"
  "CGA" => "R"
  "TGC" => "C"
  "CAA" => "Q"
  ⋮     => ⋮

### Solução iterativa

Uma forma natural de resolver o probloema seria iterar sobre todas as possibilidades de ORFs, como faz a função abaixo.

In [107]:
function encontre_orfs(seq)
    orfs = []
    seqlen = length(seq)

    # iterar sobre todos os possíveis códons de início
    for i in 1:seqlen-2
        start_codon = sample_seq[i:i+2]

        # se start_codon é realmente um códon de início, construa a proteína
        if genetic_code[start_codon] == "M"
            println("Códon de início encontrado na posição ", i, ".")
            protein = []
            found_stop = false
            # agora iteramos de 3 em 3, como um ribossomo
            for j in i:3:seqlen-2
                codon = seq[j:j+2]
                aa_residue = genetic_code[codon]
                
                # se acharmos um códon de parada, achamos um ORF!
                if aa_residue == "*"
                    println("Códon de parada na posição ", j, ".\n")
                    push!(orfs, join(protein))
                    found_stop = true
                    break
                end
                push!(protein, aa_residue)
            end
            if !found_stop println("Nenhum códon de parada encontrado.") end
        end
    end
    
    return orfs
end

encontre_orfs (generic function with 1 method)

In [108]:
orfs = encontre_orfs(sample_seq)
println("\nORFs encontrados: ", orfs)

Códon de início encontrado na posição 5.
Códon de parada na posição 8.

Códon de início encontrado na posição 25.
Códon de parada na posição 67.

Códon de início encontrado na posição 31.
Códon de parada na posição 67.

Códon de início encontrado na posição 76.
Nenhum códon de parada encontrado.

ORFs encontrados: Any["M", "MGMTPRLGLESLLE", "MTPRLGLESLLE"]


#### Notas sobre a função `encontre_orfs`

A função `append!` adiciona elementos de um conjunto de elementos, um a um. Se quisermos adicionar a `String` toda de uma vez temos que usar a `push!`, que serve para adicionar um único elemento.

In [71]:
vetor = []
println("Tipo de []: ", typeof(vetor))
println("O vetor está vazio? ", isempty(vetor))
append!(vetor, "letras")
push!(vetor, "letras")
println(vetor)

Tipo de []: Vector{Any}
O vetor está vazio? true
Any['l', 'e', 't', 'r', 'a', 's', "letras"]


A indexação em Julia começa do 1 e inclui o último índice:

In [72]:
println(vetor[2:3])
println("estringue"[1:3])

Any['e', 't']
est


### Expressões regulares

A forma mais sucinta de se abordar esse problema, contudo, é provavelmente usando expressões regulares.

Comecemos definindo uma função para traduzir uma sequência de nucleotídeos:

In [114]:
function translate(seq)
    protein = [] 
    for i in 1:3:length(seq)-2
        push!(protein, genetic_code[seq[i:i+2]])
    end
    return join(protein)
end

translate (generic function with 1 method)

In [115]:
translate(sample_seq)

"SHVANSGYMGMTPRLGLESLLE*A*MIRVASQ"

A função de encontrar ORFs pode então ser escrita:

In [140]:
function encontre_orfs_regex(seq)
    orfs = eachmatch(r"(ATG(?:.{3})*?)(?:TAA|TGA|TAG)", seq, overlap=true)
    return map(s -> translate(s[1]), orfs)
end

encontre_orfs_regex (generic function with 1 method)

In [141]:
encontre_orfs_regex(sample_seq)

3-element Vector{String}:
 "M"
 "MGMTPRLGLESLLE"
 "MTPRLGLESLLE"

A expressão regular seleciona todas as substrigs que começam com ATG e terminam com TAA, TGA, ou TAG. Entre os dois códons, requere-se ainda que um grupo de três caracteres quaisquer, `(?:.{3})`, se repita, de forma que o número de 

`s -> translate(s[1])` é uma notação para definir uma função que toma um objeto `s` e retorna `translate(s[1])`. Nesse caso, `s` é um objeto do tipo `RegexMatch`, e por isso fazemos `s[1]`, para extrair o primeiro grupo que capturamos com a busca, uma `String`, antes de passá-lo à `translate`. 

In [138]:
regex_match = match(r"(ATG(?:.{3})*?)(?:TAA|TGA|TAG)", sample_seq[6:end])
println(regex_match.match)
println(regex_match[1])

ATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAA
ATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAA
